In [42]:
import heapq
import pandas as pd
import sys
import numpy as np

In [48]:
class Node:
    # initialises the Node class with the following attributes
    def __init__(self, node_id):
        self.id = node_id
        self.adjacent = {}
        self.distance = sys.maxsize
        self.visited = False
        self.parent_node = None
    
    # adds an adjacent node with a given weight to the interaction
    def add_adjacent(self, adjacent_node, weight = 1):
        self.adjacent[adjacent_node] = weight
    
    def get_weight(self, adjacent_node):
        return self.adjacent[adjacent_node]
    
    # sets the distance from the source node to a given node
    def set_distance(self, dist):
        self.distance = dist

    # sets the parent node
    def set_parentnode(self, parent_node):
        self.parent_node = parent_node
    
    # indicates if the node has been visited by the algorithm
    def set_visited(self):
        self.visited = True
    
    # sets the proper way to compare classes
    def __le__(self, other):
        return self.distance <= other.distance
    
    def __lt__(self, other):
        return self.distance < other.distance
    
class SignedNode(Node):

    def __init__(self, node_id):
        super().__init__(node_id)
        self.distances = {1:sys.maxsize, -1:sys.maxsize}
        self.adjacent_signs = {}
        self.parent_nodes = {1: None, -1: None}
    
    def add_sign(self, adjacent_node, sign = 1):
        self.adjacent_signs[adjacent_node] = sign
    
    def get_sign(self, adjacent_node):
        return self.adjacent_signs[adjacent_node]

class Network:
    def __init__(self, sourcenodes):
        self.source_nodes = sourcenodes
        self.target_nodes = []
        self.net = pd.DataFrame()
    
    def add_targetnodes(self, node_id):
        self.target_nodes.append(node_id)
    
class Graph:

    def __init__(self):
        self.nodes_dict = {}
        self.num_nodes = 0
        self.subnetwork = ()
    
    def __iter__(self):
        return iter(self.nodes_dict.values())

    # creates a Node object and appends it to the nodes dictionary in the Graph class
    # it also counts up the number of nodes in the graph
    def add_node(self, node_id):
        self.num_nodes =+ 1
        new_node = Node(node_id)
        self.nodes_dict[node_id] = new_node
        return new_node
    
    def get_node(self, node_id):
        if node_id in self.nodes_dict:
            return self.nodes_dict[node_id]
        else:
            print('The node {} could not be found in the network!'.format(node_id))
            return None
    
    # adds an edge between a source node and a target node, and a weight ruling that interaction
    # If a Node class is not available for that node name, it creates one.
    # last, it adds to the adjacent attribute from the source Node class a dictionary item containing the target node id and the weight.
    def add_edge(self, source, target, weight = 1):
        if source not in self.nodes_dict:
            self.add_node(source)
        if target not in self.nodes_dict:
            self.add_node(target)
        
        self.nodes_dict[source].add_adjacent(self.nodes_dict[target], weight)
    
    def read_sif(self, path):
        network_df = pd.read_csv(path, sep='\t')
        for index, row in network_df.iterrows():
            self.add_edge(row['source'], row['target'], row['weight'])
    
    def dijkstra_solver(self, start_node):
        # Set the distance for the start node to zero 
        start_node.distance = 0

        # Priority queue, it gets the node with the smallest distance from the list within the unvisited nodes
        unvisited_queue = [(node.distance, node) for node in self]
        heapq.heapify(unvisited_queue)

        while len(unvisited_queue):
            # gets the node with the smallest distance 
            clostest_node = heapq.heappop(unvisited_queue)
            current_node = clostest_node[1]
            current_node.set_visited()

            # checks the adjacent nodes, doesn't recheck if visited.
            # it relaxes the edges by checking if there are quicker paths to a node than the ones already explored
            for adjacent_node in current_node.adjacent:
                if adjacent_node.visited:
                    continue
                new_dist = current_node.distance + current_node.get_weight(adjacent_node)
                
                if new_dist < adjacent_node.distance:
                    adjacent_node.set_distance(new_dist)
                    adjacent_node.parent_node = current_node

            # Rebuild the quieue
            # empty the list
            while len(unvisited_queue):
                heapq.heappop(unvisited_queue)
            # resets the queue
            unvisited_queue = [(node.distance, node) for node in self if not node.visited]
            heapq.heapify(unvisited_queue)
    
    # it gets a node, gets the name of the parent and appends it to a tuple. 
    def shortest_path(self, node, path):
        if node.parent_node:
            path.append(node.parent_node.id)
            self.shortest_path(node.parent_node, path)
        return
    
    def get_subnetwork(self, start_ids, downstream_nodes):
        self.subnetwork = Network(start_ids)
        for start_id in start_ids:
            start_node = self.get_node(start_id)
            self.dijkstra_solver(start_node)
            # 
            for node in downstream_nodes:
                target = self.get_node(node)
                if target is None: continue
                if node not in self.subnetwork.target_nodes: self.subnetwork.add_targetnodes(node) 
                path = [target.id]
                self.shortest_path(target, path)
                path.reverse()
                for i in range(0, len(path)-1):
                    new_row = pd.Series({'source': path[i], 'int': 'N', 'target': path[i+1]})
                    self.subnetwork.net = pd.concat([self.subnetwork.net, new_row.to_frame().T])
            self.subnetwork.net.drop_duplicates(inplace=True)


class SignedGraph(Graph):
    def __init__(self):
        super().__init__()

    def add_signednode(self, node_id):
        self.num_nodes =+ 1
        new_node = SignedNode(node_id)
        self.nodes_dict[node_id] = new_node
        return new_node
    
    def add_signededge(self, source, target, weight):
        if source not in self.nodes_dict:
            self.add_signednode(source)
        if target not in self.nodes_dict:
            self.add_signednode(target)

        self.nodes_dict[source].add_adjacent(self.nodes_dict[target], abs(weight))
        self.nodes_dict[source].add_sign(self.nodes_dict[target], np.sign(weight))
        
    def read_signed_sif(self, path):
        network_df = pd.read_csv(path, sep='\t')
        for index, row in network_df.iterrows():
            self.add_signededge(row['source'], row['target'], row['weight'])
    
    def double_label(self, start_node):
        start_node.distances =  {1: 0, -1: sys.maxsize}

        # Priority queue, it gets the node with the smallest distance from the list within the unvisited nodes
        unvisited_queue = [(min(node.distances[1], node.distances[-1]), node) for node in self]
        heapq.heapify(unvisited_queue)

        while len(unvisited_queue):
            # gets the node with the smallest distance 
            clostest_node = heapq.heappop(unvisited_queue)
            current_node = clostest_node[1]
            current_node.set_visited()
            
            # checks the adjacent nodes, doesn't recheck if visited.
            # it relaxes the edges by checking if there are quicker paths to a node than the ones already explored
            for adjacent_node in current_node.adjacent:
                # if adjacent_node.visited:
                #     continue
                
                adj_sign = current_node.get_sign(adjacent_node)



                if adjacent_node.distances[adj_sign] > (current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == 1:
                        adjacent_node.distances[adj_sign] = current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)
                        adjacent_node.parent_nodes[adj_sign] = current_node

                
                if adjacent_node.distances[-adj_sign] > (current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == 1:
                        adjacent_node.distances[-adj_sign] = current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)
                        adjacent_node.parent_nodes[-adj_sign] = current_node
                
                if adjacent_node.distances[-adj_sign] > (current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == -1:
                        adjacent_node.distances[-adj_sign] = current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)
                        adjacent_node.parent_nodes[-adj_sign] = current_node
                
                if adjacent_node.distances[adj_sign] > (current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == -1:
                        adjacent_node.distances[adj_sign] = current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)
                        adjacent_node.parent_nodes[adj_sign] = current_node

            # Rebuild the quieue
            # empty the list
            # while len(unvisited_queue):
            #     heapq.heappop(unvisited_queue)
            # resets the queue
            unvisited_queue = [(node.distance, node) for node in self if not node.visited]
            heapq.heapify(unvisited_queue)

    def shortest_pos_path(self, node, path):
        if node.parent_nodes[1]:
            print(node.parent_nodes[1])
            path[1].append(node.parent_nodes[1].id)
            self.shortest_pos_path(node.parent_nodes[1], path)
        return

    def shortest_neg_path(self, node, path):
        if node.parent_nodes[-1]:
            print(node.parent_nodes[-1])
            path[-1].append(node.parent_nodes[-1].id)
            self.shortest_neg_path(node.parent_nodes[-1], path)
        return
    
    def get_signed_subnetwork(self, start_ids, downstream_nodes):
        self.subnetwork = Network(start_ids)
        for start_id in start_ids:
            start_node = self.get_node(start_id)
            self.double_label(start_node)
    
            for node in list(downstream_nodes.keys()):
                target = self.get_node(node)
                if target is None: continue
                if node not in self.subnetwork.target_nodes: self.subnetwork.add_targetnodes(node) 
                path = {1:[target.id], -1:[target.id]}
                if np.sign(downstream_nodes[node]) == 1:
                    self.shortest_pos_path(target, path)
                    if len(path[1]) < 2: continue
                    path[1].reverse()
                    print(path[1])
                    for i in range(0, len(path[1])-1):
                        print(i)
                        print(self.get_node(path[1][i]).adjacent.id)
                        sign = self.get_node(path[1][i]).get_sign(path[1][i+1])
                        new_row = pd.Series({'source': path[1][i], 'int': sign, 'target': path[1][i+1]})
                        self.subnetwork.net = pd.concat([self.subnetwork.net, new_row.to_frame().T])
                elif np.sign(downstream_nodes[node]) == -1:
                    self.shortest_neg_path(target, path)
                    if len(path[-1]) < 2: continue
                    path[-1].reverse()
                    print(path[-1])
                    for i in range(0, len(path[-1])-1):
                        print(i)
                        print(path[-1][i])
                        sign = self.get_node(path[-1][i]).get_sign(path[-1][i+1])
                        new_row = pd.Series({'source': path[-1][i], 'int': sign, 'target': path[-1][i+1]})
                        self.subnetwork.net = pd.concat([self.subnetwork.net, new_row.to_frame().T])

            self.subnetwork.net.drop_duplicates(inplace=True)


In [59]:
pd.read_csv('network_collectri.sif', sep='\t')

,source,weight,target
0,CALM1,-1,TRPC1
1,CALM3,-1,TRPC1
2,CALM2,-1,TRPC1
3,CAV1,1,TRPC1
4,MDFI,-1,TRPC1
...,...,...,...
81749,NFKB,1,hsa-miR-143-3p
81750,AP1,1,hsa-miR-206
81751,NFKB,1,hsa-miR-21-5p
81752,NFKB,1,hsa-miR-224-5p


In [60]:
g = SignedGraph()
g.read_signed_sif('toy_net.sif')
downstream_nodes = {'d':1, 'e':-1}

In [61]:
g.get_node('a').get_sign()

TypeError: SignedNode.get_sign() missing 1 required positional argument: 'adjacent_node'

In [62]:
start_node = g.get_node('a')
g.double_label(start_node)

In [22]:
g.nodes_dict

{'a': <__main__.SignedNode at 0x7fad7a46c8e0>,
 'b': <__main__.SignedNode at 0x7fad7a7e8520>,
 'c': <__main__.SignedNode at 0x7fad7a563880>,
 'f': <__main__.SignedNode at 0x7fad7a5883d0>,
 'd': <__main__.SignedNode at 0x7fad7a7e8550>,
 'e': <__main__.SignedNode at 0x7fad7a589660>}

In [63]:
for i in ['a','b','c','d']:
    print(g.nodes_dict[i].visited)

True
True
True
True


In [64]:
for i in ['a','b','c','d']:
    if g.nodes_dict[i].parent_nodes[-1] is not None:
        print(g.nodes_dict[i].id, g.nodes_dict[i].parent_nodes[-1].id)

d b


In [65]:
for i in ['a','b','c','d']:
    if g.nodes_dict[i].parent_nodes[1] is not None:
        print(g.nodes_dict[i].id, g.nodes_dict[i].parent_nodes[1].id)

b a
c b


In [39]:
g.nodes_dict['f'].parent_nodes[1].id

'a'

In [132]:
test = None
if test:
    print(test)

AttributeError: 'NoneType' object has no attribute 'id'

In [168]:
test = {1:['d','c','f','a'], -1:['c','d','e','f']}
test[1].reverse()
test[1][0]

'a'

In [67]:
g.get_signed_subnetwork(downstream_nodes = {'d':1}, start_ids='a')
g.subnetwork.net

""


In [ ]:

    
def double_label(aGraph, start_node):
    start_node.distances =  {1: 0, -1: sys.maxsize}

    # Priority queue, it gets the node with the smallest distance from the list within the unvisited nodes
    unvisited_queue = [(min(node.distances[1], node.distances[-1]), node) for node in self]
    heapq.heapify(unvisited_queue)

    while len(unvisited_queue):
        # gets the node with the smallest distance 
        clostest_node = heapq.heappop(unvisited_queue)
        current_node = clostest_node[1]
        current_node.set_visited()
        new_dist = {}
        # checks the adjacent nodes, doesn't recheck if visited.
        # it relaxes the edges by checking if there are quicker paths to a node than the ones already explored
        for adjacent_node in current_node.adjacent:
            if adjacent_node.visited:
                continue
            
            adj_sign = adjacent_node.get_sign()
            
            new_dist = current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)

            if adjacent_node.distances[adj_sign] > (current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == 1:
                    adjacent_node.distances[adj_sign] = new_dist
                    adjacent_node.parent_nodes[adj_sign] = current_node
            
            if adjacent_node.distances[-adj_sign] > (current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == 1:
                    adjacent_node.distances[-adj_sign] = new_dist
                    adjacent_node.parent_nodes[-adj_sign] = current_node
            
            if adjacent_node.distances[-adj_sign] > (current_node.distances[adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == -1:
                    adjacent_node.distances[-adj_sign] = new_dist
                    adjacent_node.parent_nodes[-adj_sign] = current_node
            
            if adjacent_node.distances[adj_sign] > (current_node.distances[-adj_sign] + current_node.get_weight(adjacent_node)) and adj_sign == -1:
                    adjacent_node.distances[adj_sign] = new_dist
                    adjacent_node.parent_nodes[adj_sign] = current_node

        # Rebuild the quieue
        # empty the list
        while len(unvisited_queue):
            heapq.heappop(unvisited_queue)
        # resets the queue
        unvisited_queue = [(node.distance, node) for node in self if not node.visited]
        heapq.heapify(unvisited_queue)
        
